In [1]:
import numpy as np
import pickle
import pandas as pd
from tensorflow.keras import models
from tensorflow.keras.layers import Dense, Input, Dropout, LSTM, Masking, Bidirectional, Concatenate
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau

In [2]:
X = np.asarray(pd.read_csv("data/diabetes/X_enc.csv", header=None))
X_val = np.asarray(pd.read_csv("data/diabetes/X_val_enc.csv", header=None))
X_test = np.asarray(pd.read_csv("data/diabetes/X_test_enc.csv", header=None))
Y = np.asarray(pd.read_csv("data/diabetes/Y.csv", header=None))
Y_val = np.asarray(pd.read_csv("data/diabetes/Y_val.csv", header=None))
Y_test = np.asarray(pd.read_csv("data/diabetes/Y_test.csv", header=None))

In [3]:
print(Y.shape, Y_val.shape)

(6000, 1) (2000, 1)


In [4]:
X_discharge_disposition = pickle.load(open("data/diabetes/X_discharge_disposition_enc.csv", "rb"))
X_val_discharge_disposition = pickle.load(open("data/diabetes/X_val_discharge_disposition_enc.csv", "rb"))
X_test_discharge_disposition = pickle.load(open("data/diabetes/X_test_discharge_disposition_enc.csv", "rb"))

In [5]:
print(X_discharge_disposition.shape)

(6000, 5, 44)


In [6]:
X_medical_speciality = pickle.load(open("data/diabetes/X_medical_speciality_enc.csv", "rb"))
X_val_medical_speciality = pickle.load(open("data/diabetes/X_val_medical_speciality_enc.csv", "rb"))
X_test_medical_speciality = pickle.load(open("data/diabetes/X_test_medical_speciality_enc.csv", "rb"))

In [7]:
print(X_medical_speciality.shape)

(6000, 3, 51)


In [8]:
X_diagnoses = pickle.load(open("data/diabetes/X_diagnoses_enc.csv", "rb")).reshape(6000, 10, -1)
X_val_diagnoses = pickle.load(open("data/diabetes/X_val_diagnoses_enc.csv", "rb")).reshape(2000, 10, -1)
X_test_diagnoses = pickle.load(open("data/diabetes/X_test_diagnoses_enc.csv", "rb")).reshape(2000, 10, -1)

In [9]:
print(X_diagnoses.shape)

(6000, 10, 3174)


In [10]:
def model():
    x = Input(shape=(65,))
    
    x_d = Input(shape=(10, 3174))
    x_dis = Input(shape=(5, 44))
    x_m = Input(shape=(3, 51))
    
    rnn_d = LSTM(40)(x_d)
    rnn_dis = LSTM(20)(x_dis)
    rnn_m = LSTM(20)(x_m)
    
    conc = Concatenate()([x, rnn_d, rnn_dis, rnn_m])

    dense = Dense(400, activation='relu', name="dense")(conc)
    dense = Dense(400, activation='relu', name="dense2")(dense)
    dense = Dense(200, activation='relu', name="dense3")(dense)
    dense = Dense(100, activation='relu', name="dense4")(dense)
    y = Dense(1, activation='sigmoid', name="y")(dense)

    model = models.Model([x, x_d, x_dis, x_m], y, name="model")
    model.compile(optimizer='Adam', loss="binary_crossentropy", metrics=['acc'])

    return model

In [11]:
m = model()

In [12]:
m.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 10, 3174)]   0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 5, 44)]      0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 3, 51)]      0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 65)]         0                                            
______________________________________________________________________________________________

In [13]:
Y.shape

(6000, 1)

In [14]:
cbs = []
cbs.append(EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1))
cbs.append(ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2))        
m.fit([X, X_diagnoses, X_discharge_disposition, X_medical_speciality], Y, shuffle=True, epochs=100, \
      validation_data=([X_val, X_val_diagnoses, X_val_discharge_disposition, X_val_medical_speciality], Y_val), \
      callbacks=cbs)

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 4 array(s), but instead got the following list of 3 arrays: [array([[0.03523807, 0.        , 0.08222217, ..., 0.        , 0.        ,
        0.        ],
       [0.03279521, 0.        , 0.05465868, ..., 0.        , 0.        ,
        0.        ],
       [0.0...